Plot:

    Line plot - years X crime type

In [37]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

In [2]:
crime = pd.read_csv('data/Crime_Data_from_2010_to_2019.csv')

In [3]:
crime['Crm Cd Desc'].value_counts().head(15)

BATTERY - SIMPLE ASSAULT                                    190537
BURGLARY FROM VEHICLE                                       162174
VEHICLE - STOLEN                                            159882
THEFT PLAIN - PETTY ($950 & UNDER)                          149860
BURGLARY                                                    147695
THEFT OF IDENTITY                                           127834
INTIMATE PARTNER - SIMPLE ASSAULT                           114551
VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)     109449
ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT               92717
VANDALISM - MISDEAMEANOR ($399 OR UNDER)                     90431
THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)              88497
ROBBERY                                                      83858
THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD     74604
CRIMINAL THREATS - NO WEAPON DISPLAYED                       56650
SHOPLIFTING - PETTY THEFT ($950 & UNDER)                     4

In [4]:
topCrm = crime['Crm Cd Desc'].value_counts()[:10].index.tolist()

In [5]:
crime = crime[crime['Crm Cd Desc'].isin(topCrm)]

In [6]:
crimeDF = crime[['DATE OCC', 'AREA NAME', 'Crm Cd Desc']].copy()

In [7]:
crimeDF.loc[:, 'DATE'] = crimeDF['DATE OCC'].apply(lambda x: x.split()[0])
crimeDF.loc[:, 'DATE'] = pd.to_datetime(crimeDF['DATE'], format = '%m/%d/%Y')
crimeDF.drop(columns=['DATE OCC'], inplace=True)

In [8]:
crimeDF['DATE'] = crimeDF['DATE'].apply(lambda x: x.strftime('%Y-%m'))
crimeDF.head()

,AREA NAME,Crm Cd Desc,DATE
1,Pacific,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",2010-09
6,Central,BURGLARY FROM VEHICLE,2010-01
7,Central,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",2010-01
8,Central,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",2010-01
10,Central,BATTERY - SIMPLE ASSAULT,2010-01


In [9]:
dic = {'BATTERY - SIMPLE ASSAULT': 'Simple assault', 'BURGLARY FROM VEHICLE': 'Burglary from vehicle', 'VEHICLE - STOLEN': 'Stolen vehicle',
       'THEFT PLAIN - PETTY ($950 & UNDER)': 'Small theft (under 950$)', 'BURGLARY': 'Burglary', 'THEFT OF IDENTITY' : 'Teft of identity',
       'INTIMATE PARTNER - SIMPLE ASSAULT' :'Intimate partner assault', 'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)' : 'Vandalism (felony)',
       'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT' : 'Agravated assault', 'VANDALISM - MISDEAMEANOR ($399 OR UNDER)': 'Vandalism (misdeameanor)'
      }

In [10]:
crimeDF.replace({"Crm Cd Desc": dic}, inplace=True)
crimeDF.rename(columns={"AREA NAME": "Area", "Crm Cd Desc": "Crime", "DATE": "Date"}, inplace=True)
crimeDF.head()

,Area,Crime,Date
1,Pacific,Vandalism (felony),2010-09
6,Central,Burglary from vehicle,2010-01
7,Central,Agravated assault,2010-01
8,Central,Agravated assault,2010-01
10,Central,Simple assault,2010-01


In [11]:
crimeDF.rename(columns={"AREA NAME": "Area", "Crm Cd Desc": "Crime", "DATE": "Date"}, inplace=True)

In [12]:
crimeTypesDF = crimeDF.pivot_table(index=['Date', 'Area'], columns='Crime', aggfunc='size').rename_axis(None, axis=1)
crimeTypesDF.reset_index(level=['Date', 'Area'], inplace=True)
crimeTypesDF.fillna(0)
crimeTypesDF.head()

,Date,Area,Agravated assault,Burglary,Burglary from vehicle,Intimate partner assault,Simple assault,Small theft (under 950$),Stolen vehicle,Teft of identity,Vandalism (felony),Vandalism (misdeameanor)
0,2010-01,77th Street,88,85,89,60,160,63,92,103,46,58
1,2010-01,Central,29,13,67,24,109,55,17,49,14,15
2,2010-01,Devonshire,15,94,108,44,65,47,71,88,42,54
3,2010-01,Foothill,32,56,96,56,56,38,75,131,43,41
4,2010-01,Harbor,14,84,43,48,48,51,84,115,40,42


In [13]:
crimeTypesDF.to_csv('Processed data for plotting/Antonio/crimeDate_crimeTypes.csv')

In [88]:
crimeTypesDF = pd.read_csv('Processed data for plotting/Antonio/crimeDate_crimeTypes.csv')

In [89]:
crime_type = crimeTypesDF.copy()

def crime_line(year, area):
    
    crime_type_Area = crime_type.query('Area == @area')
    crime_type_Area = crime_type_Area[crime_type_Area['Date'].str.contains(year)]
    crimes_list = ['Agravated assault', 'Burglary','Burglary from vehicle', 'Intimate partner assault', 'Simple assault',
                   'Small theft (under 950$)', 'Stolen vehicle', 'Teft of identity','Vandalism (felony)', 
                   'Vandalism (misdeameanor)']
    crimes_to_hide = ['Agravated assault', 'Intimate partner assault', 'Teft of identity', 'Vandalism (felony)', 'Vandalism (misdeameanor)']
    
    data = [dict(type = 'scatter',
                x = crime_type_Area['Date'],
                y = crime_type_Area[crime],
                name=crime)for crime in crimes_list]
    for d in data:
        if d.get('name') in crimes_to_hide:
            d['visible'] = 'legendonly'    

    layout = dict(yaxis = dict(title = 'Count'), xaxis = dict(title = 'Date'))
    fig = go.Figure(data=data, layout=layout)
    return fig

In [90]:
crime_line('2011', 'Harbor')